In [130]:
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

def make_basis_staircase_dataset(n_samples=1000, random_state=42, coeffs=[1.0, 0.8, 0.5, 0.0, 0.0, 0.0]):
    rng = np.random.default_rng(random_state)

    # Create random bits for 6 features
    X_bits = rng.integers(0, 2, size=(n_samples, len(coeffs)))  # random 0/1

    # Calculate raw score using the given coefficients
    raw_score = np.sum(np.array(coeffs) * X_bits, axis=1)

    # Label: thresholded raw score
    y = (raw_score > 1.0).astype(int)

    # Create DataFrame
    df = pd.DataFrame(X_bits, columns=[f"f{i}" for i in range(len(coeffs))])
    df["target"] = y
    return df


In [131]:
import numpy as np
import pennylane as qml

class FarhiBasisVQC:
    def __init__(self, coeffs):
        """
        coeffs: list of feature coefficients, e.g. [1.0, 0.8, 0.5, 0.0, 0.0, 0.0]
        """
        self.coeffs = np.array(coeffs, dtype=float)
        self.num_qubits = len(self.coeffs)
        wires = list(range(self.num_qubits + 1))  # +1 for readout
        self.dev = qml.device("default.qubit", wires=wires)
        self.opt = qml.optimize.AdamOptimizer(0.1)
        self.num_layers = 1

        # Random init
        np.random.seed(0)
        self.weights = qml.numpy.array(
            (np.pi / 2) * np.random.uniform(-1, 1, size=self.num_layers * self.num_qubits),
            requires_grad=True,
        )
        self.bias = qml.numpy.array(0.0, requires_grad=True)

        self._initialize_circuit()

    def state_preparation(self, x_bin):
        """Basis encode the binary input."""
        qml.BasisEmbedding(x_bin, wires=list(range(self.num_qubits)))
        qml.PauliX(wires=self.num_qubits)

    def relationship_unitary(self, x_bin):
        for i in range(self.num_qubits):
            for j in range(i + 1, self.num_qubits):
                angle = (self.coeffs[i] * self.coeffs[j] * x_bin[i] * x_bin[j]) * np.pi / 2
    
                # Sandwich construction
                qml.CNOT(wires=[i, j])
                qml.RZ(angle, wires=j)
                qml.CNOT(wires=[i, j])


    def _initialize_circuit(self):
        @qml.qnode(self.dev, interface="autograd")
        def circuit(weights, x_bin):
            # 1) encode input
            self.state_preparation(x_bin)

            # 2) entangle features
            self.relationship_unitary(x_bin)

            # 3) variational controlled RXs into readout qubit
            for layer in range(self.num_layers):
                for j in range(self.num_qubits):
                    idx = layer * self.num_qubits + j
                    qml.ctrl(qml.RX, control=j)(-weights[idx], wires=self.num_qubits)

            # 4) measurement
            return qml.expval(qml.PauliZ(wires=self.num_qubits))

        self.circuit = circuit

    def variational_classifier(self, weights, bias, x_bin):
        return 0.5 * self.circuit(weights, x_bin) + bias

    def cost(self, weights, bias, X, Y):
        preds = qml.numpy.array([
            0.5 * (qml.numpy.tanh(self.variational_classifier(weights, bias, x)) + 1)
            for x in X
        ])
        return qml.numpy.mean((qml.numpy.array(Y) - preds) ** 2)

    def fit(self, X_train, y_train, num_epochs=50, learning_rate=0.05, batch_size=48):
        X = np.array(X_train, dtype=np.float64)
        Y = np.array(y_train)

        self.opt = qml.optimize.AdamOptimizer(learning_rate)
        for _ in range(num_epochs):
            idx = np.random.choice(len(X), min(batch_size, len(X)), replace=False)
            X_batch, Y_batch = X[idx], Y[idx]
            self.weights, self.bias = self.opt.step(
                lambda ws, b: self.cost(ws, b, X_batch, Y_batch),
                self.weights, self.bias
            )
        return self

    def predict(self, X):
        X = np.array(X, dtype=np.float64)
        return np.array([
            1 if self.variational_classifier(self.weights, self.bias, x) > 0 else 0
            for x in X
        ])


In [132]:
def evaluate_model(model, df, features, label, basis_encoding=True):
    if basis_encoding:
        # Features are already 0/1 from make_basis_staircase_dataset()
        X = df[[f"f{i}" for i in range(6)]].values
        y = df["target"].values

        # Mask features
        X_masked = np.zeros_like(X)
        for idx in features:
            X_masked[:, idx] = X[:, idx]

    else:
        # Old real-valued features from make_staircase_dataset()
        X = df[[f"f{i}" for i in features]].values
        y = df["target"].values

        # Scale real inputs
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Put into 6D format
        X_masked = np.zeros((X_scaled.shape[0], 6))
        for idx, feature_idx in enumerate(features):
            X_masked[:, feature_idx] = X_scaled[:, idx]

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_masked, y, test_size=0.2, random_state=42
    )

    # Fit and evaluate
    model.fit(X_train, y_train, num_epochs=10, learning_rate=0.1)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)
    print(f"{label:<20} | Train: {acc_train:.4f} | Test: {acc_test:.4f}")


In [134]:
# === Sanity Check ===
def run_sanity_check():
    print("\n🔍 Running sanity checks on basis_staircase_dataset...\n")
    

    base_coeffs = [1.0, 0.8, 0.5] + [0.0] * 3
    df = make_basis_staircase_dataset(n_samples=1000, random_state=42, coeffs=base_coeffs)
    # base_coeffs = [1.0, 0.0, 0.0, 0.8, 0.5, 0.0]
    # df = make_basis_staircase_dataset(n_samples=1000, coeffs=base_coeffs)

    feature_sets = [
        ([0],       "f0"),
        ([0, 1],    "f0 + f1"),
        ([0, 2],    "f0 + f2"),
        ([0, 3],    "f0 + f3 (noise)"),
        ([0, 4],    "f0 + f4 (noise)"),
        ([0, 5],    "f0 + f5 (noise)"),
        ([0, 1, 2], "f0 + f1 + f2"),
    ]

    print("=== Basis‐Encoded VQC ===")
    for features, label in feature_sets:
        coeffs = [base_coeffs[i] for i in range(6)]
        model = FarhiBasisVQC(coeffs)
        evaluate_model(model, df, features, label)

if __name__ == "__main__":
    run_sanity_check()



🔍 Running sanity checks on basis_staircase_dataset...

=== Basis‐Encoded VQC ===
f0                   | Train: 0.7400 | Test: 0.7250
f0 + f1              | Train: 0.7638 | Test: 0.7900
f0 + f2              | Train: 0.7150 | Test: 0.7500
f0 + f3 (noise)      | Train: 0.7400 | Test: 0.7250
f0 + f4 (noise)      | Train: 0.7400 | Test: 0.7250
f0 + f5 (noise)      | Train: 0.6462 | Test: 0.6050
f0 + f1 + f2         | Train: 1.0000 | Test: 1.0000


NameError: name 'FarhiAngleVQC' is not defined